# K-Nearest Neighbor

Berikut ini adalah langkah-langkah dalam menyelesaikan permasalahan klasifikasi menggunakan KNN :

### 1. Membaca data train

In [1]:
import pandas as pd

data_train_from_csv = pd.read_csv('dataTrain.csv')

data_train_from_csv.head()

,atribut 1,atribut 2,atribut 3,atribut 4,kelas
0,17.4,61.250,10.4,21.0,1
1,16.2,34.375,14.8,15.6,1
2,11.4,46.875,10.8,9.9,0
3,12.6,76.250,18.0,25.5,1
4,16.2,55.000,9.8,15.9,0


### 2. Menghitung Euclidean Distance

Dalam melakukan perhitungan jarak pada KNN dapat digunakan bebrapa teori, diantanya adalah *euclidean* dan *manhattan*. Dalam percobaan kali ini akan dilakukan dengan menggunakan teori *euclidean* dalam menghitung jaraknya.

*Euclidean distance* melakukan perhitungan jarak antara dua buah titik dalam *euclidean space*. Berikut adalah formulanya :

<img src="assets/euclidean.png">

In [2]:
import numpy as np

def euclideanDistance(data_train, data_test):
    distanceSum = 0
    for i in range(len(data_train)-1):
        distanceSum += (data_train[i]-data_test[i])**2
    return np.sqrt(distanceSum)

### 3. KNN Algorithm

KNN merupakan algoritma yang digunakan dalam melakukan memecahkan permasalahan klasifikasi, sehingga menghasilkan output diskrit. Contoh untuk output berupa diskrit adalah output yang hasilnya pasti seperti ketika menghitung 1 + 1 = 2, jawabannya bukan mendekati 2. KNN akan melakukan klasifikasi terhadap objek berdasarkan data pembelajaran yang jaraknya paling dekat dengan objek tersebut.

KNN akan bekerja berdasarkan jarak minimum dari data baru ke data training untuk menentukan tetangga terdekat. Setelah itu akan didapatkan data mayoritas sebagai hasil prediksi dari data baru tadi.

In [3]:
import operator

def kNearestNeighbor(data_train, data_test, k):
    distances = {}
    sort = {}
    neighbors = []
    vote_class = {}
  
    for i in range(len(data_train)):
        distance = euclideanDistance(data_train.iloc[i], data_test)
        distances[i] = distance
  
    sorted_distances = sorted(distances.items(), key=operator.itemgetter(1))
  
    for i in range(k):
        neighbors.append(sorted_distances[i][0])
    
    for x in range(len(neighbors)):
        class_in_datatrain = data_train.iloc[neighbors[x]][-1]
    
        if class_in_datatrain in vote_class:
            vote_class[class_in_datatrain] += 1
        else:
            vote_class[class_in_datatrain] = 1
  
    sorted_vote_class = sorted(vote_class.items(), key=operator.itemgetter(1))
  
    return sorted_vote_class[-1][0]

### 4. Menghitung akurasi

Akurasi akan didapatkan dari perbandingan hasil prediksi dengan data sebenarnya.

In [4]:
def predictionAccuracy(prediction_data, data_test):
    accurate = 0
  
    for i in range(len(prediction_data)):
        if prediction_data[i] == data_test.iloc[i][-1]:
            accurate += 1
      
    return (accurate/len(prediction_data)) * 100

### 5. Cross Validation & Tuning Parameter

**Cross Validation**

*Cross validation* merupakn metode statistik dalam melakukan evaluasi kinerja dari suatu model atau algoritma dengan melakukan pembagian data menjadi dua subset, yaitu `data pengujian` dan `data pelatihan`.

> **K-Fold Cross Validation**
K-Fold Cross Validation merupakan salah satu metode Cross validation yang bekerja dengan melipat data sebanyak K dan melakukan perulangan sebanyak K juga. Contohnya untuk k = 10:

<img src="assets/k-fold.png">

**Tuning Parameter**

Untuk mendapatkan akurasi yang terbaik saat melakukan klasifikasi di KNN, akan sangat bergantung pada nilai `K` yang kita berikan. Proses dalam mencari `K` terbaik dapat disebut sebagain *Tuning Parameter8* atau *Hyperparameter*.

In [ ]:
from statistics import mean

def crossValFtTunParam(data_train_from_csv):
    fold1 = data_train_from_csv.iloc[0:800]
    fold2 = data_train_from_csv.iloc[800:1600]
    fold3 = data_train_from_csv.iloc[1600:2400]
    fold4 = data_train_from_csv.iloc[2400:3200]
    fold5 = data_train_from_csv.iloc[3200:]
    k_range = 3
    best_k = {}
    for k in range(1, k_range):
        accuracy_crossval = []
        for i in range(1,6):
            prediction_data = []
            if i == 1:
                data_test = fold1
                data_train = pd.concat([fold2, fold3, fold4, fold5])
            elif i == 2:
                data_test = fold2
                data_train = pd.concat([fold1, fold3, fold4, fold5])
            elif i == 3:
                data_test = fold3
                data_train = pd.concat([fold1, fold2, fold4, fold5])
            elif i == 4:
                data_test = fold4
                data_train = pd.concat([fold1, fold2, fold3, fold5])
            else:
                data_test = fold5
                data_train = pd.concat([fold1, fold2, fold3, fold4])

            for x in range(len(data_test)):
                prediction_data.append(kNearestNeighbor(data_train, data_test.iloc[x], k))
                
            print('i =>', i)
            print(prediction_data)
            
            tmp_accuracy = predictionAccuracy(prediction_data, data_test)
            accuracy_crossval.append(tmp_accuracy)
            
            print('Accuracy =>', tmp_accuracy)
        
        best_k[k] = accuracy_crossval.mean()
        
    K = max(best_k.items(), key=operator.itemgetter(1))[0]
    
    print(best_k)
        
    return K, best_k[K]

### Mencari K terbaik dan Akurasinya

In [ ]:
best_k, accuracy = crossValFtTunParam(data_train_from_csv)
print('Best k : ', best_k)
print('Accuracy : ', accuracy, '%')

### Calculate Data Test CSV

In [ ]:
data_test_from_csv = pd.read_csv('dataTest.csv')

data_test_from_csv.head()

In [ ]:
data_train = data_train_from_csv[:3200]

prediction_data = []
for x in range(len(data_test_from_csv)):
    prediction_data.append(kNearestNeighbor(data_train, data_test_from_csv.iloc[x], best_k))

In [ ]:
data_test_from_csv['kelas'] = prediction_data

data_test_from_csv.to_csv('Prediksi_Tugas2AI_1301160790.csv')

### Hasil Prediksi

In [ ]:
data_hasil_from_csv = pd.read_csv('Prediksi_Tugas2AI_1301160790.csv')
data_hasil_from_csv